<a href="https://colab.research.google.com/github/elibergman/QM2515-Tarea-Nro-4/blob/main/Tarea%20Nro.%204/Tarea_4_Resoluci%C3%B3n_de_Ecuaciones_Diferenciales_de_Primer_orden_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sympy import symbols, Function, Eq, dsolve, Derivative, lambdify, sympify, parse_expr, solve
import matplotlib.pyplot as plt
import numpy as np

# METODO ANALITICO
print("METODO ANALITICO")
print()
# Definir las variables simbólicas
t = symbols('t')
y = Function('y')(t)  # Definimos y como función de t

print("Para resolver la ecuacion diferencial, debe estar igualada a 0.")
print("Ingrese la ecuación diferencial en la forma y' = f(t, y)")
print()
input_ecuacion = input("Ingrese la ecuación diferencial (ejemplo: y' - y - t): ")

# Reemplazar "y'" por "Derivative(y, t)"
input_ecuacion = input_ecuacion.replace("y'", "Derivative(y, t)")

# Convertir la entrada del usuario en una expresión de Sympy
ecuacion_sympy = parse_expr(input_ecuacion, local_dict={'y': y, 'Derivative': Derivative, 't': t})

# Crear la ecuación diferencial
ed = Eq(ecuacion_sympy, 0)

# Solicitar al usuario las condiciones iniciales
input_CI = input("Ingrese las condiciones iniciales (ejemplo: y(0)=5): ")

# Procesar la entrada de condiciones iniciales
partes_CI = input_CI.split('=')
valor_t = sympify(partes_CI[0].split('(')[1].split(')')[0])  # Extraer el valor de t
lado_derecho = sympify(partes_CI[1])
CI = {y.subs(t, valor_t): lado_derecho}

# Resolver la ecuación diferencial con las condiciones iniciales
sol_implicita = dsolve(ed, y, ics=CI)

# Intentar obtener la solución explícita
sol_explicita = solve(sol_implicita, y)

# Convertir la solución simbólica en una función numérica
funcion_num = lambdify(t, sol_explicita, modules=['numpy'])

a = float(input("Ingrese el valor inicial para el grafico: "))
b = float(input("Ingrese el valor final para el grafico: "))

# Rango de valores de t
t_vals = np.linspace(a, b, 200)

solucion = sol_explicita[0] if isinstance(sol_explicita, list) else sol_explicita

# Convertir la solución simbólica en una función numérica si aún no lo es
funcion_solucion = lambdify(t, solucion, modules=['numpy'])

# Evaluar cada valor en t_vals y almacenar en y_vals
y_vals = [funcion_solucion(valor_t) for valor_t in t_vals]

# Ahora y_vals contiene los valores de la función evaluados en cada punto de t_vals



print()
#METODO DE EULER
print("METODO DE EULER")
print()


# Definir la ecuación diferencial
y_prima = symbols('y_prima')

#Cambio de variable para poder operar
cambio_var = sympify(input_ecuacion.replace("Derivative(y, t)", "y_prima"))

ecuacion_euler = sympify(-1 * cambio_var + y_prima)

print(f"Ahora resolveremos la ecuacion {ecuacion_euler} con el metodo de Euler ")

# Pedir al usuario que ingrese dos tamaños de paso
h1 = float(input("Introduce el primer tamaño del paso (h1), por ejemplo, 0.1: "))
h2 = float(input("Introduce el segundo tamaño del paso (h2), por ejemplo, 0.2: "))


def ecu_euler(y, t):
    # Utilizamos lambdify para convertir la expresión de Sympy en una función numérica
    funcion_numerica = lambdify([y_prima, 'y', 't'], ecuacion_euler, modules=['numpy'])
    return funcion_numerica(y, y, t)

# Resto del código para el método de Euler
def aplicar_euler(h):
    t = np.arange(a, b, h)
    y = np.zeros_like(t)
    y[0] = 1  # Condición inicial
    for i in range(1, len(t)):
        y[i] = y[i-1] + h * ecu_euler(y[i-1], t[i-1])
    return t, y

# Calcular y graficar para cada paso h
t1, y1 = aplicar_euler(h1)
t2, y2 = aplicar_euler(h2)



print()

#METODO DE RUNGE-KUTTA

print("METODO DE RUNGE-KUTTA" )
print()
print("Ahora, resolveremos la ecuacion con el metodo de Runge-Kutta")



def RK(f, y0, t0, tf, h):

    t = np.arange(t0, tf + h, h)
    y = np.zeros(len(t))
    y[0] = y0
    for i in range(1, len(t)):
        k1 = f(t[i-1], y[i-1])
        k2 = f(t[i-1] + 0.5 * h, y[i-1] + 0.5 * h * k1)
        k3 = f(t[i-1] + 0.5 * h, y[i-1] + 0.5 * h * k2)
        k4 = f(t[i-1] + h, y[i-1] + h * k3)
        y[i] = y[i-1] + (h / 6) * (k1 + 2 * k2 + 2 * k3 + k4)
    return t, y

# Ejemplo de uso
def ecu_RK(t, y):
   funcion_numerica = lambdify([y_prima, 'y', 't'], ecuacion_euler, modules=['numpy'])
   return funcion_numerica(y, y, t)

# Condiciones iniciales y parámetros
y0 = 1  # Condición inicial
t0 = a  # Tiempo inicial
tf = b  # Tiempo final
h = float(input("Ingrese un valor de h para el metodo de Runge-Kutta: "))  # Tamaño del paso

# Calcular la solución
t3, y3 = RK(ecu_RK, y0, t0, tf, h)



print()
print()
# Crear la gráfica
plt.figure()
plt.plot(t_vals, y_vals, label='Función y(t) metodo analitico')
plt.plot(t1, y1, label=f'Aproximación por Método de Euler (h1={h1})')
plt.plot(t2, y2, label=f'Aproximación por Método de Euler (h2={h2})')
plt.plot(t3, y3, label=f'Aproximación por Método de Runge-Kutta (h={h})')
plt.xlabel('t')
plt.ylabel('y')
plt.title('Gráfica de los 3 metodos utilizados, Analitico, euler y RK')
plt.grid(True)
plt.legend()
plt.show()
print()
print("La solucion analitica es:", sol_explicita)
